<a href="https://colab.research.google.com/github/saeu5407/CGDClassifier/blob/main/src/gan_classifier/video_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/github/CGDClassifier/src/cnn_classifier

/content/drive/MyDrive/github/CGDClassifier/src/cnn_classifier


In [3]:
!pip install onnx
!pip install onnxruntime

     |████████████████████████████████| 12.8 MB 7.6 MB/s 
     |████████████████████████████████| 5.2 MB 6.2 MB/s 


In [4]:
import glob
import onnxruntime
import torch
from PIL import Image
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import cv2 as cv
import os
import numpy as np
import pandas as pd

In [5]:
def make_video(video_path, ort_session):

    cap = cv.VideoCapture(video_path)
    index = 0
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    model_writer = cv.VideoWriter('{}'.format(video_path.replace(".mp4","_pred.mp4")), fourcc, 20, (640, 480))
    cat_list = ['drink', 'hang', 'lookout', 'phone', 'sleep', 'study', 'study_boring']
    index_max = None
    result_df = []

    def softmax(x):

        x = x - np.max(x)
        f_x = np.exp(x) / np.sum(np.exp(x))
        return f_x

    if index_max == None:
        index_max = 99999999999

    while True:

        ret, input_img = cap.read()

        if index%100 == 0:
          print(">>> {} Index Save".format(index))

        if not ret or (index == index_max):
          print('>>> {} End'.format(os.path.basename(video_path)))
          cap.release()
          model_writer.release()
          cv.destroyAllWindows()
          pd.DataFrame(result_df, columns = ['index', 'category', 'drink', 'hang', 'lookout', 'phone', 'sleep', 'study', 'study_boring']).to_csv("{}".format(video_path.replace(".mp4","_pred.csv")))
          break

        image_tensor = F.to_tensor(input_img)
        image_tensor = F.resize(image_tensor, (260, 260))
        image_tensor = F.normalize(image_tensor, (0.5,0.5,0.5), (0.5,0.5,0.5))
        image_tensor = image_tensor.unsqueeze(0)

        ort_inputs = {ort_session.get_inputs()[0].name : image_tensor.numpy()}
        ort_outs = ort_session.run(None, ort_inputs)
        pred_list = softmax(ort_outs[0][0]).round(2)
        pred = pred_list.argmax()
        pred_list = list(pred_list)
        pred_per = round(pred_list[int(pred)] * 100, 2)

        text1 = 'Index : {}'.format(index)
        text2 = 'Outputs : {}'.format(pred_list)
        text3 = 'Predict : {}, {}%'.format(cat_list[int(pred)], pred_per)

        frame = cv.resize(input_img, (640, 480))
        cv.putText(frame, text1, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255))
        cv.putText(frame, text2, (10, 40), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255))
        cv.putText(frame, text3, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255))

        result_list = [index, cat_list[int(pred)]]
        result_list.extend(pred_list)
        result_df.append(result_list)

        model_writer.write(frame)
        index += 1

In [6]:
model_path = glob.glob(os.path.join(os.getcwd().split(os.path.join(".","src").replace(".",""))[0], "model", "*.onnx"))[0]
video_path_list = glob.glob(os.path.join(os.getcwd().split(os.path.join(".","src").replace(".",""))[0], "datasets", "test", "*.mp4"))
ort_session = onnxruntime.InferenceSession(model_path)

for video_path in video_path_list:

    print(os.path.basename(video_path))
    make_video(video_path, ort_session)

test4.mp4
>>> 0 Index Save
>>> 100 Index Save
>>> 200 Index Save
>>> 300 Index Save
>>> 400 Index Save
>>> 500 Index Save
>>> 600 Index Save
>>> 700 Index Save
>>> 800 Index Save
>>> 900 Index Save
>>> 1000 Index Save
>>> 1100 Index Save
>>> 1200 Index Save
>>> 1300 Index Save
>>> 1400 Index Save
>>> 1500 Index Save
>>> 1600 Index Save
>>> 1700 Index Save
>>> 1800 Index Save
>>> 1900 Index Save
>>> 2000 Index Save
>>> 2100 Index Save
>>> 2200 Index Save
>>> 2300 Index Save
>>> 2400 Index Save
>>> 2500 Index Save
>>> 2600 Index Save
>>> 2700 Index Save
>>> 2800 Index Save
>>> 2900 Index Save
>>> 3000 Index Save
>>> 3100 Index Save
>>> test4.mp4 End
